In [ ]:
# Add 'hide code' button
from IPython.display import HTML
HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
Der Code für dieses IPython-Notebook ist zur leichteren Lesbarkeit standardmäßig ausgeblendet. Um den Code ein-/auszuschalten, klicken Sie auf <a href="javascript:code_toggle()">hier</a>.''')

In [ ]:
# interactive widgets
from __future__ import print_function
from tqdm import tqdm_notebook as tqdm
# from tqdm import notebook # progress bar
from ipywidgets import interactive
from ipywidgets import interact, widgets


# data wrangling
from data_wrangling import *
import pandas as pd
import numpy as np
import datetime, inspect
from os import path


# plotting
from plot_functions import * # own plot functions from plot_functions.py
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("darkgrid")
import plotly.express as px
import plotly.offline
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.graph_objects as go


# settings
pd.set_option('mode.chained_assignment', None) # TO DO: Deal with SettingWithCopyWarning
clrs = get_config()['color'] # four base colors for visualization

# pio.renderers.default = "jupyterlab"

# data import
dba = DBAccess() # connect to db and load new data
data = dba.select('sensor_data') # flagged = 0, attendance = 1
dirty = dba.select('sensor_data_dirty') # data inclusive of flagged data (attendance = 1)
person = dba.select('person') # dataframe containing information to every participant
occupation_table = dba.select('occupation_period')
mal = pd.read_csv('../database/Malfunction.csv') # table with malfunction timeframes
not_o = pd.read_csv('../database/not_occupied.csv') # table with information about missing timeframes
df_tso = dba.select('turn_sensor_occupation').astype({'occupation_period_id':'int64'})

# drop duplicates
data = data.drop_duplicates('log_sensor_id')
dirty = dirty.drop_duplicates('log_sensor_id')

# enrich person with cingle and couple information
df_person = living_situation_person(person)

import warnings
warnings.filterwarnings('ignore') # change 'ignore' to 'once' to see a warning once

# **Vacancy – No Vacancy**
----

**Autoren**: Roman Studer, Simon Luder, Lukas Gehrig

Diese Arbeit befasst sich mit der Auswertung von Sensordaten welche im Rahmen des Projektes "vacancy no-vacancy" durch die ETH Zürich gesammelt wurden.
Für die Dauer eines Jahres steht seit dem Sommer 2019 das Mock-Up einer Wohnung auf der Dachterrasse des HIL-Gebäudes der ETH Zürich. Personen unterschiedlicher sozialer Gruppen wohnen für je eine Woche im Mock-Up.

Im Mock-up sind vier beweglichen Elemente (Drehwand, Lampe Drehwand, Drehschrank, Lampe Aussenwand) prototypisch eingesetzt. Sensoren messen die Bewegungshäufigkeit, die Zeitpunkte sowie die jeweils gewählten Winkel sämtlicher beweglicher Bauteile. Türen und Schubladen sind ebenfalls mit Sensoren ausgestattet, welche eine Interaktion mit den Elementen binär festhält (ON/OFF oder OPEN/CLOSED). Die Auswertung der erhobenen Daten gibt Aufschlüsse über das dynamische Zusammenspiel zwischen Menschen und Wohnung. Mehr zum Projekt finden Sie auf der [Website](https://arch.ethz.ch/news-und-veranstaltungen/lehre-forschung/vacancy-novacancy.html) der ETH Zürich.

Der Fokus dieser Datenanalyse liegt auf den Drehelementen. Diese werden durch den Raum beschränkt. Die Ausgangslage ist bei jedem Drehelement 0°, wobei eine Drehung im Uhrzeigersinn erfolgt. Somit können die Bewohner den Raum individuell nach Ihren Bedürfnissen gestalten.

Die Datenanalyse beginnt mit der Überprüfung der Drehsensorwerte, um diese allenfalls zu korrigieren oder zu ignorieren. Anschließend folgt eine Übersicht über die Teilnehmer bezüglich Wohnsituation, Alter und Geschlecht in Form von verschiedenen Barplots. Der erste Teil der eigentlichen Analyse untersucht die Anzahl Bewegungen der Drehelemente und anderen Sensoren. Es folgt eine Analyse der simultanen Bewegungen von Drehwand und Drehschrank. Zu guter Letzt befindet sich eine Analyse über die Positionen der Drehelemente.

Die Häufigkeits- und Positionsanalyse untersucht auch Verhaltensunterschiede der Mock-Up Bewohner bezüglich Wohnsituation, Alter und Geschlecht.

![Lageplan Mock-up](layout.PNG)

## Datenbereinigung
----

Die Drehsensoren liefern einen Anfangswert, Endwert und den dazugehörigen Zeitstempel, sobald ein Drehelement bewegt wurde. Somit haltet ein Drehelement seine Position, bis ein neuer Eintrag des entsprechenden Sensors eintrifft. Der Wertebereich ist von 0° bis zum Maximum des entsprechenden Drehelementes.

Die folgenden zwei Darstellungen haben den Zweck, die Sensorwerte auf Unregelmäßigkeiten und Fehler zu untersuchen. Weitere Zeitintervalle, welche für die Analyse ignoriert werden müssen, wie Spezialanlässe oder ein unbesetztes Mock-Up, sind hier nicht ersichtlich, da lediglich die Auswirkung der entschiedenen Behandlungsmöglichkeiten aufzeigt werden soll. Dies ist ein unabdingbarer Schritt um zu verhindern, dass die Hauptanalyse zeitlich und positionsbedingt nicht zu stark verfälscht wird.

Die Darstellungen zeigen interaktiv die Stellungen der vier Drehsensoren über die gesamte Projektdauer. Die x-Achse gibt Datum, oder beim Heranzoomen auch Urzeit, an. y-Achse gibt den jeweiligen Winkel des Sensors in Grad an. Mit der Maus kann ein Teilstück der Grafik angewählt werden (Click & Drag) sowie der Sensor angezeigt werden (Hover). Um wider zurück auf die Gesamtansicht zu kommen, oben links auf den Knopf "Alle" klicken. In der oberen rechten Ecke der Grafik kann mittels eines Dropdown alle oder einzelne Sensoren ausgewählt werden. Bei einzelner Ansicht stellen die zwei orangen Linien die Wertegrenzen des jeweiligen Sensors dar.

### Sensordaten Drehelemente vor Bereinigung

Wenn die Sensoren die Werte korrekt liefern, ist der Anfangswert des neuen Eintrages in der ersten Darstellung identisch mit dem Endwert des letzten Eintrags desselben Sensors. Da die Drehelemente die Stellung des letzten Endwertes halten, bis ein neuer Eintrag eintrifft, werden die Einträge interpoliert. D. h. in der Annahme, dass die Daten korrekt sind, sollten nur vertikale und horizontale Linien ersichtlich sein. Mithilfe dieser Information und des Wertebereiches eines Sensors können Anomalien in den Daten identifiziert werden.

Es bieten sich folgende Behandlungsmöglichkeiten an:
- Verschiebung eines Sensors nach oben oder unten innerhalb eines bestimmten Zeitintervalls
- Das Zeitintervall für alle weiteren Analysen ignorieren
- Das Zeitintervall für die Positionsanalyse ignorieren, jedoch für die Häufigkeitsanalyse berücksichtigen.

In [ ]:
# Lineplot of Sensordata before exclusion of flagged values
p_rotary(sensor_states)

### Sensordaten Drehelemente nach Bereinigung

Diese Darstellung zeigt die Auswirkungen der oben genannten Behandlungsmöglichkeiten aufzuzeigen. Hierbei wurden folgende Entscheide getroffen:
- Es werden lediglich die Endwerte berücksichtigt
- Bewegungen unter einem in 'config.yml' definierten Grenzwert werden nicht als Bewegung akzeptiert. Somit reduziert sich die Anzahl Bewegungen und die Position wird als unverändert wahrgenommen.
- Daten, welche für die Positionsanalyse ignoriert werden, jedoch nicht für die Häufigkeitsanalyse, werden hier nicht aufgezeigt (obwohl dies Teil der Analyse sind)
- Extremwerte ausserhalb des Wertebereiches eines Drehsensors werden auf das Minimum/Maximum +/- die definierte Toleranz zurückgestuft.

In [ ]:
# Lineplot of Sensordata without flagged and thus usable data
p_rotary(prep_rotary_sensors)

## Teilnehmerverteilung

Es folgt eine Analyse der Teilnehmerverteilung. Dies hat den Zweck, eine Übersicht über die Teilgenommenen Personen und Personengruppen zu erhalten bezüglich Wohnsituation, Alter und Geschlecht.

Diese Darstellung zeigt, wie viele Singles und Paare an der Studie teilgenommen haben.
Paar bedeutet, dass zwei Personen während der Woche im Mock-Up gewohnt haben. Diese haben nicht zwingend eine Beziehung.

In [ ]:
single_couple(df_person)

Diese Darstellung zeigt die Alterskategorien, in welchen sich die Teilnehmer zum Zeitpunkt der Studie befanden.

In [ ]:
age_dist(person)

Diese Darstellung zeigt die Verteilung der Geschlechter der Teilnehmer.

In [ ]:
sex_dist(df_person)

Diese Darstellung zeigt die Verteilung der Geschlechter der Teilnehmer pro Alterskategorie.

In [ ]:
sex_age_dist(df_person)

Da einige Paare das gleiche Geschlecht haben und dies für die weiteren Analysen von Bedeutung ist, zeigt diese Darstellung die Zusammensetzung der Geschlechter der Paare pro Alterskategorie. Auf der y-Achse werden die Anzahl Personen der Altersgruppe (nicht Paare) abgebildet.

In [ ]:
couples_sex_age(df_person)

Diese Darstellung zeigt die Geschlechterverteilung von Singles nach Alter.

In [ ]:
singles_sex_age_dist(df_person)

Diese Darstellung zeigt die Geschlechterverteilung von gleichgeschlechtrigen Paaren nach Alter.

In [ ]:
ss_couples_sex_age_dist(df_person)

Diese Darstellung zeigt die Berufsverteilung der Teilnehmer.

In [ ]:
profession_dist(person)

## Bewegungsanalyse
Dieser Abschnitt behandelt die Häufigkeit von Bewegungen von interaktiven Elementen im Mock-Up. Ein besonderer Fokus wird auf die vier Drehsensoren gelegt (Siehe Gebäudeplan in der Einleitung). 

### Nutzung nach Jahreszeit, Wochentag & Tageszeit
Je nach Jahreszeit, Wochentag oder Tageszeit kann es zu unterschiedlichem Verhalten kommen. Unterschiede zwischen den Drehsensoren sind hier besonders interessant.

#### Durchschnittliche tägliche Bewegungen auf ein Jahr verteilt

In [ ]:
t_monthly, p_monthly = p_monthly()

p_monthly

**Lesebeispiel:**
Die Grafik zeigt die durchschnittlichen täglichen Interaktionen auf Monate gruppiert. Unter der Überschrift "Alle Sensoren" sind die durchschnittlichen, täglichen Bewegungen pro Monat für alle Sensoren aufgelistet. Bsp: ein Wert von 12 heisst, das an diesem Monat pro Tag im Schnitt 12 mal mit dem entsprechenden Sensor interagiert wurde. Mit dem Dropdown-Menü oben rechts kann auf ein Sensor fokussiert werden. Diese Grafik bezieht alle Bewegungen mit ein die nicht durch eine Fehlfunktion oder einen Unterbruch als unbrauchbar gekennzeichnet wurden. Bewegungen die also ausserhalb des offiziellen Projektzeitraums entstanden sind, sind nicht enthalten. Beachten Sie dass es möglich sein kann, dass Monate nicht vertreten sind da noch keine Daten verfügbar sind.

#### Durchschnittliche tägliche Bewegungen auf eine Woche verteilt

In [ ]:
t_weekly, p_weekly = p_weekly()

In [ ]:
p_weekly

**Lesebeispiel** Die Grafik zeigt die durchschnittlichen täglichen Interaktionen pro Wochentag. Unter der Überschrift "Alle Sensoren" sind die durchschnittlichen, täglichen Bewegungen pro Wochentag für alle Sensoren aufgelistet. Am Mo. (Montag) wurde die Drehwand (blau) X-Mal bewegt. Betrachten Sie dazu die Zahl oberhalb des Balkens. Die einzelnen Werte sind errechnet durch: (Interaktionen / Gültige Stunden) und auf einen Tag hochgerechnet worden.

#### Durchschnittliche stündliche Bewegungen auf 24 Stunden verteilt

In [ ]:
t_daily, p_daily = p_daily()

In [ ]:
p_daily

**Lesebeispiel:**
Die Grafik zeigt die durchschnittlichen täglichen Interaktionen auf einzelne Stunden gruppiert. Bsp: ein Wert von 1.5 heisst, das zu dieser Uhrzeit im Schnitt 1.5 mal mit dem entsprechenden Sensor interagiert wird.

* Alle Sensoren: Hier wurden alle Sensoraufzeichnungen innerhalb der Wohnbesetzungen mitgezählt welche durch direkte Interaktion mit Kontakt und Drehsensoren entstanden sind.
* Drehwand: Hier wurden alle Sensoraufzeichnungen innerhalb der Wohnbesetzungen mitgezählt welche durch direkte Interaktion mit dem Sensor "Drehwand" entstanden sind.
* Drehschrank: Hier wurden alle Sensoraufzeichnungen innerhalb der Wohnbesetzungen mitgezählt welche durch direkte Interaktion mit dem Sensor "Drehschrank" entstanden sind.
* Lampe Drehwand: Hier wurden alle Sensoraufzeichnungen innerhalb der Wohnbesetzungen mitgezählt welche durch direkte Interaktion mit dem Sensor "LampeDrehwand" entstanden sind.
* Lampe Aussenwand: Hier wurden alle Sensoraufzeichnungen innerhalb der Wohnbesetzungen mitgezählt welche durch direkte Interaktion mit dem Sensor "LampeAussenwand" entstanden sind.

#### Sensoren pro Woche, im Detail

In [ ]:
sens_weekly()

**Lesebeispiel:** Die Grafik zeigt die X-te Woche des Experimentes. Insgesamt sieben Barplots zeigen die Bewegungen der Drehsensoren während der Besetzungsperiode als einzelne Wochentage. Die y-Achse pro Plot zeigt die Anzahl Bewegungen pro Stunde, koloriert als Sensortyp. Die x-Achse zeigt um welche Stunde es sich handelt. 0 Uhr ist Mitternacht. Die Skala endet um 23 Uhr. Oberhalb der Grafik kann anhand des Sliders "Besetzungsperiode" eine der Besetzungsperioden angewählt werden. Informationen zur Woche, sprich Alter, Geschlecht und Identifaktionsnummer der Personen sind unterhalb angezeigt. Ebenfalls ist es möglich mit dem Dropdown-Menü "Sensorgruppe" eine von vier Sensorgruppen auszuwählen. Die Sensoren wurden aufgeteilt in die Bereiche "Drehsensoren, Küche, Schubladen & Schränke und Türen. Mit der Maus kann per click&drag herangezoomt werden. Um die Grafik zurückzusetzen, klicken sie oben rechts auf das Symbol (Haus) "Reset Axes".

Sensordaten die aufgrund einer Fehlfunktion oder eines anderen Grundes nicht als gültig markiert wurden, sind nicht in der Grafik enthalten.

## Sensorbewegungen nach Kategorien
In diesem Abschnitt können die Unterschiede von Sensorbewegungen anhand der unterschiedlichen Alters- und Geschlechtsgruppen betrachtet werden. Die folgenden Grafiken verwenden auch Sensordaten, deren Positionsdaten nicht verwendet werden können. Das bedeutet das die Bewegung eines Drehsensors verwendet wird auch, wenn der Datenpunkt eine fehlerhafte Position aufgrund eines Sensorfehlers aufweist. Dies kann mit einer Anpassung des CSV's "Malfunction.csv" abgeändert werden. Bitte nehmen Sie dazu die Bedienungsanleitung zur Hand.

### Bewegungen nach Altersgruppe

In [ ]:
def_sensor_age_week()

**Lesebeispiel:** Diese Grafik zeigt die Unterschiede von Anzahl Bewegungen (auf eine Woche normiert) pro Altersgruppe von Singles und Paaren. Mit den Dropdown-Menüs "Sensor1" und "Sensor2" kann jegliche Kombination von Sensoren dargestellt werden. Bitte beachten Sie das die Berechnung eines neuen Plots etwas Zeit in Anspruch nimmt.

In [ ]:
def_sensor_age_day()

**Lesebeispiel** Die obere Grafik stellt die Anzahl Bewegungen eines Sensores aufgeteilt nach Wochentage dar. Auf der X-Achse sind die jeweiligen Wochentage (Mo. - Fr.) dargestellt und in die jeweiligen Altersgruppen aufgeteilt. Fehlt für den Sensor informatinen zu einer der vier Altersgruppen, wird diese nicht dargestellt. Die Y-Achse zeigt die totale Anzahl Bewegungen für den jeweiligen Tag. Mit hilfe des Dropdown-Menüs "Sensor" oberhalb der Grafik kann einer der verfügbaren Sensoren angewählt werden. Bitte beachten Sie dass die Berechnung einige Sekunden dauern kann. 

### Bewegungen nach Geschlecht

In [ ]:
def_sensor_sex_week()

**Lesebeispiel:** Diese Grafik zeigt die Unterschiede von Anzahl Bewegungen (auf eine Woche normiert) pro Geschlechtsgruppe von Singles und Paaren. Mit den Dropdown-Menüs "Sensor1" und "Sensor2" kann jegliche Kombination von Sensoren dargestellt werden. Bitte beachten Sie das die Berechnung eines neuen Plots etwas Zeit in Anspruch nimmt.

In [ ]:
def_sensor_sex_day()

**Lesebeispiel** Die obere Grafik stellt die Anzahl Bewegungen eines Sensors aufgeteilt nach Wochentage dar. Auf der x-Achse sind die jeweiligen Wochentage (Mo. – Fr.) dargestellt und auf die jeweiligen Geschlechter aufgeteilt. Fehlt für den Sensor Informationen zu einer der Geschlechtsgruppen, wird diese nicht dargestellt. Die y-Achse zeigt die totale Anzahl Bewegungen für den jeweiligen Tag. Mithilfe des Dropdown-Menüs "Sensor" oberhalb der Grafik kann einer der verfügbaren Sensoren angewählt werden. Bitte beachten Sie, dass die Berechnung einige Sekunden dauern kann.

## Simultane Bewegungen

Die Drehwand kann in Verbindung mit dem Drehschrank genutzt werden, um temporäre Räumlichkeiten zu erstellen, welche zum Beispiel den Bereich der Küche oder den Schlafbereich vom Rest der Wohnung trennen. Da dazu eine Bewegung von sowohl der Drehwand als auch des Drehschrankes benötigt wird, wird im folgenden Kapitel geprüft wie oft eine "simultane" Bewegung stattgefunden hat. Zusätzlich wird betrachtet, ob die Lampe oberhalb der Drehwand ebenfalls zusammen mit der Drehwand bewegt wird. Die Definition von "zusammen bewegt" ist subjektiv. Die Wahl einer sinnvollen Zeitspanne ist dem Betrachter überlassen und kann angepasst werden. Standard ist auf 60 Sekunden festgelegt. Diese Zeitspanne lässt auf einen starken Zusammenhang zwischen der Bewegung der Drehwand und der Bewegung des Drehschrankes schließen. Eine Anpassung dieser Zeitspanne kann in der Konfigurationsdatei vorgenommen werden. Bitte folgen Sie dazu den im Bedienungshandbuchs vorgegebenen Schritten.

Die Daten in diesem Kapitel sind nicht normiert. Es handelt sich somit um totale und nicht angepasste Angaben. Die verschiedenen Verhältnisse von Personen unterschiedlichen Geschlechts oder Alters muss beachtet werden.

### Simultane Bewegung von Drehwand und Drehschrank pro Tag

In [ ]:
sim_filtered = filter_sim(60, filter_mal(usable=1))

plot_sim_movement_total(sim_filtered, mal, not_o)

**Lesebeispiel** Rötlich eingefärbte Bereiche der Grafik (oberhalb) stellen Perioden dar in denen Daten aufgrund von Fehlfunktionen fehlten oder wenn kein Partizipanten im Gebäude war. Zwischen dem 30. November 2019 und dem 30. Januar 2019 lieferte der Sensor der Drehwand keine verlässlichen Daten. Der Drehschrank wurde weiterhin bedient. Da die Daten zum Drehschrank für besagten Zeitraum fehlen, können keine Angaben zur simultanen Benutzung von Drehschrank und Drehwand getätigt werden. Die grosse Lücke ab dem 14. Mai 2019 ist auf den Ausbruch des Virus Covid-19 und des darauf folgenden Unterbruchs des Projektes zurückzuführen.


### Simultane Bewegung von Drehwand und Drehschrank verteilt durch den Tag

In [ ]:
event_hour = sim_move_hour(sim_filtered)

plot_sim_move_hour(event_hour, "Drehwand", "Drehschrank" )

Die obere Grafik stellt die totale Anzahl simultaner Bewegungen der Sensoren "Drehwand" und "Drehschrank" für jede Stunde dar. Bitte beachten Sie, dass 12 Uhr Nachts 0 Uhr ist.

### Simultane Bewegung von Drehwand und Drehschrank verteilt auf Wochentag

In [ ]:
rotary_wd = pd.DataFrame(sim_filtered.groupby(sim_filtered.time_of_event.dt.weekday).count().iloc[:,1]).reset_index()
rotary_wd = rotary_wd.rename(columns={'time_of_event':'weekday','log_sensor1':'count'})

g = sns.catplot(data=rotary_wd,x='weekday', y='count', kind="bar", height= 6,aspect=2.5, color='#2D92D6')
g.set_xticklabels(["Mo.", "Di.","Mi.","Do.","Fr.","Sa.", "So."])
g.set(title='Anzahl simultaner Bewegungen von Drehsensor und Drehwand \n pro Wochentag',
      xlabel='Wochentag',
      ylabel='Anzahl Bewegungen')
plt.show()

Der Plot beschreibt die totale Anzahl simultaner Bewegungen der Sensoren "Drehsenor" und "Drehwand".

### Simultane Bewegung von Drehwand und Drehschrank unterteilt in Altersgruppen und Wohnsituationen

In [ ]:
data_age, data_status = sim_move_age_situation(person=person, data = sim_filtered)
plot_sim_age_situation(data_age, data_status)

Die Anzahl simultaner Bewegungen wurde ebenfalls auf die verschiedenen Altersgruppen und Single und Couple aufgeteilt.

### Simultane Bewegung von Drehwand und Drehlampe
Zusätzlich wird betrachtet, ob die Drehwand auch simultan mit der Drehlampe bewegt wurde.

In [ ]:
sim_filtered = filter_sim(get_config()['turn_sensors']['Simultane_Bewegung'], data)

event_hour = sim_move_hour(sim_filtered)

plot_sim_move_hour(event_hour, "Drehwand", "LampeDrehwand")

In [ ]:
rotary_wd = pd.DataFrame(sim_filtered.groupby(sim_filtered.time_of_event.dt.weekday).count().iloc[:,1]).reset_index()
rotary_wd = rotary_wd.rename(columns={'time_of_event':'weekday','log_sensor1':'count'})

g = sns.catplot(data=rotary_wd,x='weekday', y='count', kind="bar", height= 6, aspect=2, color='#2D92D6')
g.set_xticklabels(["Mo.", "Di.","Mi.","Do.","Fr.","Sa.", "So."])
g.set(title='Anzahl simultaner Bewegungen von Drehwad und LampeDrehwand  \n pro Wochentag',
      xlabel='Wochentag',
      ylabel='Anzahl Bewegungen')
plt.show()

Die oberen Grafiken zeigen die totale Anzahl simultaner Bewegungen von Drehwand und der Lampe oberhalb der Drehwand durch den Tag in Stunden und durch die Woche in Tagen an.

## Positionen der Drehelemente

### Beschreibung und Entscheidungen zu den Plots

- Die Positionen der Sensoren werden als Histogramm dargestellt. Hierbei wird ein gewichtetes Histogramm verwendet, wobei die Dauer, wie lange ein Sensor in der jeweiligen Position verharrt, das Gewicht darstellt. Zeitperioden, welche entweder aus nicht korrigierbaren Sensorwerten gemäss 'Malfunction.csv' besteht und die Abwesenheit gemäss 'inhabitants.csv' der Testperson oder allfällige besondere Anlässe gemäss 'not_occupied.csv' werden für diese Analyse herausgenommen und die Dauer der Sensorzustände entsprechend angepasst. Ein Balken im Histogramm umfasst der in der Konfiguration 'config.yml' definierten Drehwinkel. Die y-Achse beinhaltet die durchschnittliche Verweildauer in Tagen pro Woche, wobei die Werte nicht normalisiert wurden. Somit beträgt die Summe immer weniger als eine Woche, wobei diese abhängig ist von den nicht berücksichtigten Zeiten wegen Sensorfehlern und anderen speziellen Ereignissen.

Beispiel Drehwand:
- 1. Eintrag: 2 min $\approx$ 50°
- 2. Eintrag: 10 min $\approx$ 25°
- 3. Eintrag: 3 min $\approx$ 50°

Somit ist der Balken bei $\approx$ 25° (10 min) doppelt so hoch wie der Balken bei $\approx$ 50° (2 + 3 min)

- Als Positionswerte werden lediglich die Endwerte der Einträge berücksichtigt.

- Einträge der Drehelemente die ausserhalb der Grenzwerte +/- der definierten Toleranz liegen, werden auf die Minimal- und Maximalwerte +/- Toleranz korrigiert. Somit sind auch kleine Werte ausserhalb der Grenzen (z.B. kleine Minuswerte) möglich.

- Positionskorrekturen gemäss 'corr.csv' werden hierbei berücksichtigt.

- Bei Zeitintervallen von Fehlfunktionen gemäss Position und besonderen Anlässen wird der letzte Eintrag des entsprechenden Sensors bis zum Anfangszeitpunkt des Intervalls gewichtet. Am Ende einer solchen Zeitperiode wird erst der erste Eintrag danach berücksichtigt. Somit entfällt eine allfällige Position, die in den Intervallen eingestellt, danach aber nicht oder später verändert wurde.

- Sensorwertveränderungen, welche nicht als Änderungen eingestuft wurden (z. B. zu kleine oder zu langsame Bewegung des Sensors), werden bei dieser Analyse nicht berücksichtigt.

- Bei der Analyse nach Alter und Geschlecht wird zwischen Paaren und Singles unterschieden.

### Alle Teilnehmer

In [ ]:
sensors_all = prep_rotary_weights_assignment(df_tso)

plot_turn_weights_all(sensors_all)

## Singles & Paare

In [ ]:
sensors_living = prep_rotary_weights_assignment(df_tso, 'living')

plot_turn_weights_living(sensors_living)

## Geschlecht

Paare und Singles werden, werden hierbei separat Ausgewiesen.
Paare mit zwei verschiedenen Geschlechtern werden nicht berücksichtigt, da die Aktionen eines jeweiligen Geschlechts nicht vom Anderen getrennt werden kann.

In [ ]:
sensors_sex = prep_rotary_weights_assignment(df_tso, 'sex')

plot_turn_weights_sex(sensors_sex, couples = False)

In [ ]:
plot_turn_weights_sex(sensors_sex, couples = True)

## Alter

In [ ]:
sensors_age = prep_rotary_weights_assignment(df_tso, 'age')

Paare und Singles werden, werden hierbei separat Ausgewiesen. 
Bei Paaren, bei welchen beide Teilnehmer nicht in derselben Alterskategorie liegen, werden hierbei nicht berücksichtigt.

In [ ]:
plot_turn_weights_age(sensors_age, couples = False)

In [ ]:
plot_turn_weights_age(sensors_age, couples = True)

# Note
Dieses Notebook kann durch die Datei "config.yml" zu einem gewissen Grad konfiguriert werden. Sollte es zum Beispiel zu Anpassungen bei den Experimentteilnehmern/teilnehmerinnen oder bei Sensorwerten kommen, können diese Anpassungen mit Hilfe der Betriebsanleitung angepasst werden. Kontaktieren Sie dafür die verantwortliche Person.